# Table of Contents
1. Creating UCCSD ansatz circuit for LiH
2. Optimizing circuit
3. Mapping circuit to nearest neighbor graph
-----
## 1. Creating UCCSD Ansatz Circuit for LiH

In [18]:
import sys
sys.path.append('..')
from fqc import uccsd, util
import numpy as np

In [19]:
theta_vector = [np.random.rand() * 2 * np.pi for _ in range(8)]
print(theta_vector)

[4.390931267983523, 1.613765232774535, 1.828499266551241, 1.413023655385834, 4.590193762449681, 2.1368319952943162, 3.8121927073707904, 3.3907205042570796]


In [20]:
circuit = uccsd.get_uccsd_circuit('LiH', theta_vector)
circuit.draw(output='text', line_length=4000)

In [21]:
util.circuitutil.get_max_pulse_time(circuit)

3241.900000000011

In [22]:
unoptimized_unitary = util.get_unitary(circuit)

## 2. Optimizing circuit

In [23]:
circuit = util.circuitutil.optimize_circuit(circuit)

In [24]:
circuit.draw(output='text', line_length=4000)

In [25]:
util.circuitutil.get_max_pulse_time(circuit)

2779.200000000008

In [27]:
optimized_unitary = util.get_unitary(circuit)
print(np.array_equal(optimized_unitary, unoptimized_unitary))

True


## 3. Mapping circuit to nearest neighbor graph

In [28]:
coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(2, 2)

In [29]:
mapped_circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)

In [30]:
mapped_circuit.draw(output='text', line_length=4000)

In [31]:
util.circuitutil.get_max_pulse_time(mapped_circuit)

2986.700000000009

In [ ]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [ ]:
import numpy as np
import os, sys, math
from datetime import datetime
sys.path.append('..'); import config

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [ ]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 1000
decay =  max_iterations / 2
convergence = {'rate':0.02, 'max_iterations':max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay}
reg_coeffs = {}

In [ ]:
N = 4
connected_qubit_pairs = util.circuitutil.get_nearest_neighbor_coupling_list(2, 2, directed=False)
H0 = hamiltonian.get_H0(N, d, connected_qubit_pairs)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d)
U = transmon_gate(optimized_unitary, d)

In [ ]:
total_time = 50.3
steps = int(total_time * 20)
uks, U_f = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path=data_path)

In [ ]:
# allow 2000 iterations-->total_time is much smaller :(

d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 2000
decay =  max_iterations / 2
convergence = {'rate':0.02, 'max_iterations':max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay}
reg_coeffs = {}
total_time = 44.4
steps = int(total_time * 20)
uks, U_f = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path=data_path)